Transformer 模型，资源消耗非常大
- 由于每一层需要存储激活结果，所以N层网络消耗的内存是单层的N倍。

- Transformer每一层中间的前馈全连接网络的维度dff要比注意力层的dmodel大的多，所以消耗的内存更多。

- 序列长度为L的attention在时间和空间的复杂度都是O(L2)，所以一个包含64K个tokens长的序列，都将会消耗巨大的内存。

Reformer 模型解决了这些问题
- 可逆神经网络，将只需要存储一层的激活结果即可，N的因素消失了。

- 分块计算前馈全连接层，节省内存。

- 采用局部敏感哈希技术，近似计算注意力，将时空开销从O(L2)变为O(L)。

https://github.com/google/trax/tree/master/trax/models/reformer

举个例子，Transformer 中每一个注意力 Head 都是相互独立的，它们之间没有信息交流，因此谷歌最近提出的 Talking-Head 就旨在解决这个问题。

- 假定序列长度还是 64K，对于 Query 序列中的每一个 q，正常的注意力机制需要计算它和 Key 序列 64K 个元素之间的相似度，并通过 Softmax 将相似度归一化为概率。
- 矩阵乘法会计算 Query 序列所有元素与 Key 序列所有元素之间的相似度，不通过矩阵乘法而是其它方法找到与 Query 序列「最相近」的 32 个或 64 个元素，计算量就会大幅减少。
- 局部敏感哈希算法（Locality sensitive hashing），查找这些元素
    - 局部敏感哈希算法（LSH）在输入数据彼此类似时，它们有很大概率映射后的哈希是一样的；而当输入数据彼此不同，它们映射后的哈希值相等概率极小。
    - LSH 算法根据局部敏感哈希函数族将类似的数据映射到相同的桶（Bucket），Query 和 Key 会先通过哈希桶排序，相似的 q 与 k 会落在相同的桶内，因此直接在桶内执行注意力计算就能逼近完整的注意力机制。

原始链接：https://github.com/lucidrains/reformer-pytorch